In [69]:
import numpy as np
import pandas as pd

def parse_complete_data():
    """解析图片中的完整数据集"""
    # 从图片中提取的所有数据
    data = [
        [0.4123, 0.1966, 0.3911, 2],
        [0.4382, 0.303, 0.2588, 0],
        [0.5329, 0.2813, 0.1859, 0],
        [0.1817, 0.197, 0.6213, 2],
        [0.4514, 0.2454, 0.3033, 0],
        [0.5623, 0.2407, 0.197, 1],
        [0.2726, 0.2315, 0.4959, 0],
        [0.2092, 0.2871, 0.5037, 1],
        [0.4884, 0.232, 0.2796, 0],
        [0.2535, 0.269, 0.4775, 2],
        [0.311, 0.2947, 0.3943, 2],
        [0.4859, 0.2841, 0.23, 0],
        [0.5938, 0.1852, 0.221, 2],
        [0.3001, 0.2604, 0.4395, 0],
        [0.522, 0.2754, 0.2025, 0],
        [0.1608, 0.2496, 0.5897, 1],
        [0.3753, 0.2234, 0.4013, 2],
        [0.2901, 0.2979, 0.412, 1],
        [0.6058, 0.1953, 0.1989, 2],
        [0.6733, 0.1933, 0.1333, 0],
        [0.5442, 0.2831, 0.1727, 0],
        [0.5769, 0.1985, 0.2245, 2],
        [0.2397, 0.2197, 0.5406, 0],
        [0.234, 0.3019, 0.464, 2],
        [0.5116, 0.2406, 0.2478, 0],
        [0.5137, 0.2298, 0.2565, 0],
        [0.4974, 0.2572, 0.2454, 1],
        [0.3072, 0.286, 0.4068, 2],
        [0.4573, 0.1795, 0.3632, 1],
        [0.4119, 0.2332, 0.3549, 0],
        [0.4877, 0.2161, 0.2962, 1],
        [0.1954, 0.2556, 0.549, 2],
        [0.5156, 0.2663, 0.218, 2],
        [0.5471, 0.2724, 0.1804, 0],
        [0.3821, 0.2386, 0.3793, 1],
        [0.1474, 0.3197, 0.5329, 2],
        [0.5624, 0.2509, 0.1868, 1],
        [0.4993, 0.2421, 0.2586, 1],
        [0.2691, 0.2341, 0.4968, 1],
        [0.5599, 0.2251, 0.2149, 0],
        [0.2241, 0.35, 0.4259, 1]
    ]
    return pd.DataFrame(data, columns=['class0', 'class1', 'class2', 'result'])

def evaluate_predictions(df, threshold_single=0.5, threshold_gap=0.15, threshold_gap_2=0.1, threshold_sum=0.8):
    """评估预测结果"""
    results = []
    correct = 0
    total = len(df)
    single_predictions = 0
    dual_predictions = 0
    
    for idx, row in df.iterrows():
        probs = [row['class0'], row['class1'], row['class2']]
        true_label = row['result']
        
        # 获取排序后的概率和对应的类别索引
        sorted_indices = np.argsort(probs)[::-1]
        sorted_probs = np.array(probs)[sorted_indices]
        
        max_prob = sorted_probs[0]
        second_prob = sorted_probs[1]
        last_prob = sorted_probs[-1]
        prob_gap = max_prob - second_prob
        
        # 确定预测策略
        if (max_prob >= 0.35) and (second_prob >= 0.35) and (max_prob <= 0.4) and (second_prob <= 0.4):
            predictions = [1]
            pred_type = "draw 特殊处理1"
            single_predictions += 1
            print("draw 特殊处理1: ", probs)
        elif (max_prob >= 0.35) and (second_prob >= 0.35) and last_prob < 0.2:
            predictions = [1]
            pred_type = "draw 特殊处理1"
            single_predictions += 1
            print("draw 特殊处理1: ", probs)
        elif probs[1] >=0.35:
            predictions = [1]
            pred_type = "draw 特殊处理2"
            single_predictions += 1
            print("draw 特殊处理2: ", probs)
        elif second_prob >= 0.28 and second_prob <= 0.32 and np.abs(second_prob - last_prob) <= 0.01:
            predictions = [1]
            pred_type = "draw 特殊处理3"
            single_predictions += 1
            print("draw 特殊处理3: ", probs)
        elif max_prob >= 0.49 and max_prob <= 0.51:
            predictions = [1]
            pred_type = "draw 特殊处理4"
            single_predictions += 1
            print("draw 特殊处理4: ", probs)
        elif (max_prob + second_prob) >= threshold_sum:
            predictions = [sorted_indices[0], sorted_indices[1]]
            pred_type = "双重预测"
            dual_predictions += 1
            print("双重预测: ", probs)
        elif max_prob >= threshold_single and prob_gap >= threshold_gap:
            predictions = [sorted_indices[0]]
            pred_type = "单一预测"
            single_predictions += 1
            print("单一预测: ", probs)

        else:
            predictions = [sorted_indices[0]]
            pred_type = "默认单一预测"
            single_predictions += 1
            print("默认单一预测: ", probs)
        
        # 检查预测是否正确
        is_correct = true_label in predictions
        if is_correct:
            correct += 1
            
        results.append({
            'index': idx + 1,
            'true_label': true_label,
            'predictions': predictions,
            'prediction_type': pred_type,
            'max_prob': max_prob,
            'second_prob': second_prob,
            'is_correct': is_correct
        })
    
    accuracy = correct / total
    
    return {
        'detailed_results': results,
        'summary': {
            'total_samples': total,
            'correct_predictions': correct,
            'accuracy': accuracy,
            'single_predictions': single_predictions,
            'dual_predictions': dual_predictions
        }
    }

# 执行分析
df = parse_complete_data()
evaluation = evaluate_predictions(df)

# 打印总体结果
summary = evaluation['summary']
print("\n总体分析结果:")
print(f"总样本数: {summary['total_samples']}")
print(f"正确预测数: {summary['correct_predictions']}")
print(f"最终准确率: {summary['accuracy']:.2%}")
print(f"单一预测数量: {summary['single_predictions']}")
print(f"双重预测数量: {summary['dual_predictions']}")

# 打印详细的预测分布
detailed_results = pd.DataFrame(evaluation['detailed_results'])
prediction_type_stats = detailed_results['prediction_type'].value_counts()
print("\n预测类型分布:")
print(prediction_type_stats)

# 计算每种预测类型的准确率
print("\n各预测类型的准确率:")
for pred_type in prediction_type_stats.index:
    type_results = detailed_results[detailed_results['prediction_type'] == pred_type]
    type_accuracy = type_results['is_correct'].mean()
    print(f"{pred_type}: {type_accuracy:.2%}")

draw 特殊处理1:  [0.4123, 0.1966, 0.3911]
默认单一预测:  [0.4382, 0.303, 0.2588]
双重预测:  [0.5329, 0.2813, 0.1859]
双重预测:  [0.1817, 0.197, 0.6213]
默认单一预测:  [0.4514, 0.2454, 0.3033]
双重预测:  [0.5623, 0.2407, 0.197]
draw 特殊处理4:  [0.2726, 0.2315, 0.4959]
draw 特殊处理4:  [0.2092, 0.2871, 0.5037]
默认单一预测:  [0.4884, 0.232, 0.2796]
默认单一预测:  [0.2535, 0.269, 0.4775]
默认单一预测:  [0.311, 0.2947, 0.3943]
默认单一预测:  [0.4859, 0.2841, 0.23]
双重预测:  [0.5938, 0.1852, 0.221]
默认单一预测:  [0.3001, 0.2604, 0.4395]
单一预测:  [0.522, 0.2754, 0.2025]
双重预测:  [0.1608, 0.2496, 0.5897]
默认单一预测:  [0.3753, 0.2234, 0.4013]
draw 特殊处理3:  [0.2901, 0.2979, 0.412]
双重预测:  [0.6058, 0.1953, 0.1989]
双重预测:  [0.6733, 0.1933, 0.1333]
双重预测:  [0.5442, 0.2831, 0.1727]
双重预测:  [0.5769, 0.1985, 0.2245]
单一预测:  [0.2397, 0.2197, 0.5406]
默认单一预测:  [0.234, 0.3019, 0.464]
单一预测:  [0.5116, 0.2406, 0.2478]
单一预测:  [0.5137, 0.2298, 0.2565]
draw 特殊处理4:  [0.4974, 0.2572, 0.2454]
默认单一预测:  [0.3072, 0.286, 0.4068]
draw 特殊处理1:  [0.4573, 0.1795, 0.3632]
默认单一预测:  [0.4119, 0.2332, 0.35

In [6]:
import numpy as np
import pandas as pd


def parse_complete_data():
    """解析图片中的完整数据集"""
    # 从图片中提取的所有数据
    data = [
        [0.2172, 0.2375, 0.5452],
        [0.5721, 0.2391, 0.1888],
        [0.2304, 0.2433, 0.5263],
        [0.5567, 0.2393, 0.2040],
        [0.2482, 0.2576, 0.4942],
        [0.5740, 0.2332, 0.1928]
    ]
    return pd.DataFrame(data, columns=['class0', 'class1', 'class2'])

def evaluate_predictions(df, threshold_single=0.5, threshold_gap=0.15, threshold_gap_2=0.1, threshold_sum=0.8):
    """简化版本的评估预测结果"""
    results = []
    
    # 遍历数据框每一行进行预测
    for idx, row in df.iterrows():
        probs = [row['class0'], row['class1'], row['class2']]
        
        # 获取排序后的概率和对应的类别索引
        sorted_indices = np.argsort(probs)[::-1]
        sorted_probs = np.array(probs)[sorted_indices]
        
        max_prob = sorted_probs[0]
        second_prob = sorted_probs[1]
        last_prob = sorted_probs[-1]
        prob_gap = max_prob - second_prob
        
        # 确定预测策略
        if (max_prob >= 0.35) and (second_prob >= 0.35) and (max_prob <= 0.4) and (second_prob <= 0.4):
            predictions = [1]
            pred_type = "draw 特殊处理1"
        elif (max_prob >= 0.35) and (second_prob >= 0.35) and last_prob < 0.2:
            predictions = [1]
            pred_type = "draw 特殊处理1"
        elif probs[1] >= 0.35:
            predictions = [1]
            pred_type = "draw 特殊处理2"
        elif second_prob >= 0.28 and second_prob <= 0.32 and np.abs(second_prob - last_prob) <= 0.01:
            predictions = [1]
            pred_type = "draw 特殊处理3"
        elif max_prob >= 0.49 and max_prob <= 0.51:
            predictions = [1]
            pred_type = "draw 特殊处理4"
        elif (max_prob + second_prob) >= threshold_sum:
            predictions = [sorted_indices[0], sorted_indices[1]]
            pred_type = "双重预测"
        elif max_prob >= threshold_single and prob_gap >= threshold_gap:
            predictions = [sorted_indices[0]]
            pred_type = "单一预测"
        else:
            predictions = [sorted_indices[0]]
            pred_type = "默认单一预测"
        
        # 保存预测结果
        results.append({
            'index': idx + 1,
            'predictions': predictions,
            'prediction_type': pred_type,
            'max_prob': max_prob,
            'second_prob': second_prob
        })
    results_df = pd.DataFrame(results)
    return results_df

df = parse_complete_data()
evaluation = evaluate_predictions(df)
print(len(evaluation))
evaluation[['index', 'predictions']]

6


,index,predictions
0,1,[2]
1,2,"[0, 1]"
2,3,[2]
3,4,[0]
4,5,[1]
5,6,"[0, 1]"
